In [1]:
import numpy as np
import os

import matplotlib.pyplot as plt

# pickle
import pickle

# tf and keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from keras.utils.vis_utils import plot_model

In [2]:
file = open("/kaggle/input/next-word-prediction-dataset/1661-0.txt", "r", encoding = "utf8")
lines = []

for i in file:
    lines.append(i)
    
data = ""

for i in lines:
    data = ' '. join(lines)
    
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“', '').replace("”", '')

data = data.split()
data = ' '.join(data)
data[:500]

"Project Gutenberg's The Adventures of Sherlock Holmes, by Arthur Conan Doyle This eBook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this eBook or online at www.gutenberg.net Title: The Adventures of Sherlock Holmes Author: Arthur Conan Doyle Release Date: November 29, 2002 [EBook #1661] Last Updated: May 20, 2019 Language: English Character set en"

In [3]:
len(data)

573660

In [4]:
#preprocessing

In [5]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

pickle.dump(tokenizer, open('tokenizer1.pkl', 'wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:10]

[142, 4680, 1, 986, 5, 125, 33, 46, 556, 2164]

In [6]:
len(sequence_data)

108958

In [7]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

8624

In [8]:
sequences = []
for i in range(3, len(sequence_data)):
    words = sequence_data[i-3:i+1]
    sequences.append(words)
    
print("The length of sequences :", len(sequences))
sequences = np.array(sequences)
sequences[:10]

The length of sequences : 108955


array([[ 142, 4680,    1,  986],
       [4680,    1,  986,    5],
       [   1,  986,    5,  125],
       [ 986,    5,  125,   33],
       [   5,  125,   33,   46],
       [ 125,   33,   46,  556],
       [  33,   46,  556, 2164],
       [  46,  556, 2164, 2165],
       [ 556, 2164, 2165,   27],
       [2164, 2165,   27,  987]])

#train and test

In [9]:
X = []
y = []

for i in sequences:
    X.append(i[0:3])
    y.append(i[3])
    
X = np.array(X)
y = np.array(y)

In [10]:
print("Data :\n", X[:10])
print()
print("Response :\n", y[:10])

Data :
 [[ 142 4680    1]
 [4680    1  986]
 [   1  986    5]
 [ 986    5  125]
 [   5  125   33]
 [ 125   33   46]
 [  33   46  556]
 [  46  556 2164]
 [ 556 2164 2165]
 [2164 2165   27]]

Response :
 [ 986    5  125   33   46  556 2164 2165   27  987]


In [11]:
y = to_categorical(y, num_classes=vocab_size)
y[:10]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

#Model Building

In [12]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=3))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))

2022-12-07 07:29:48.695385: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-07 07:29:48.696286: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-07 07:29:48.932045: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-07 07:29:48.932937: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-07 07:29:48.933740: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 3, 10)             86240     
_________________________________________________________________
lstm (LSTM)                  (None, 3, 1000)           4044000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 1000)              8004000   
_________________________________________________________________
dense (Dense)                (None, 1000)              1001000   
_________________________________________________________________
dense_1 (Dense)              (None, 8624)              8632624   
Total params: 21,767,864
Trainable params: 21,767,864
Non-trainable params: 0
_________________________________________________________________


In [15]:
checkpoint = ModelCheckpoint("nextword1.h5", monitor='loss', verbose=1, save_best_only=True)
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001))
lstm = model.fit(X, y, validation_split=0.2, epochs=100, batch_size=128, shuffle=True, callbacks=[checkpoint]).history

2022-12-07 07:31:05.966265: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 3006809344 exceeds 10% of free system memory.
2022-12-07 07:31:09.298772: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 3006809344 exceeds 10% of free system memory.


Epoch 1/100
681/681 [==============================] - 18s 22ms/step - loss: 5.3716 - val_loss: 6.1649

Epoch 00001: loss improved from inf to 5.37163, saving model to nextword1.h5
Epoch 2/100
681/681 [==============================] - 14s 21ms/step - loss: 5.1122 - val_loss: 6.2556

Epoch 00002: loss improved from 5.37163 to 5.11223, saving model to nextword1.h5
Epoch 3/100
681/681 [==============================] - 15s 21ms/step - loss: 4.8626 - val_loss: 6.4616

Epoch 00003: loss improved from 5.11223 to 4.86255, saving model to nextword1.h5
Epoch 4/100
681/681 [==============================] - 14s 21ms/step - loss: 4.6006 - val_loss: 6.7619

Epoch 00004: loss improved from 4.86255 to 4.60063, saving model to nextword1.h5
Epoch 5/100
681/681 [==============================] - 14s 21ms/step - loss: 4.3471 - val_loss: 7.1047

Epoch 00005: loss improved from 4.60063 to 4.34708, saving model to nextword1.h5
Epoch 6/100
681/681 [==============================] - 14s 21ms/step - loss: 4.

In [16]:
model = load_model("nextword1.h5")
tokenizer = pickle.load(open("tokenizer1.pkl", 'rb'))

In [17]:
def predict_next_words(model, tokenizer, text) :
    
    sequence = tokenizer.texts_to_sequences([text])
    sequence = np.array(sequence)
    
    preds = np.argmax(model.predict(sequence))
    predicted_word = ""
    
    for key, value in tokenizer.word_index.items() :
        if value == preds :
            predicted_word = key
            break
    
    return predicted_word

In [20]:
l = ["The Adventures of", "by Arthur Conan", "it is all","you will be", "0"]

for i in l :
    
    text = i
    
    if text == "0" :
        print()
        print("Execution completed...")
        break
    
    else :
        try :
            text = text.split(" ")
            text = text[-3:]
            print("Input :", i)
            print("Output : ", i, predict_next_words(model, tokenizer, text))
            print()
            
        except Exception as e :
            print("Error occured : ", e)
            continue

Input : The Adventures of
Output :  The Adventures of sherlock

Input : by Arthur Conan
Output :  by Arthur Conan doyle

Input : it is all
Output :  it is all right

Input : you will be
Output :  you will be good


Execution completed...
